In [3]:
import warnings
import os
import sys
import logging
root_dir = os.path.abspath('..')
sys.path.append(root_dir)

from models.data_container.plugin.patient_hana_ent import load

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

# --- Global style setup ---
sns.set_theme(style="white", context="notebook")
plt.rcParams.update({
    "font.size": 12,
    "axes.labelweight": "semibold",
    "axes.titleweight": "bold",
    "axes.titlesize": 14,
    "axes.labelsize": 12,
    "legend.fontsize": 11,
    "figure.facecolor": "white",
    "axes.edgecolor": "#333333",
    "axes.linewidth": 0.8,
})

In [5]:
og = load(root_dir)
demo = og[['id', 'date', 'sex', 'age']]

Loading inpatient data for 2018...
Loading inpatient data for 2019...
Loading inpatient data for 2020...
Loading inpatient data for 2021...
Loading inpatient data for 2022...
Loading inpatient data for 2023...
Loading inpatient data for 2024...
Loading outpatient data for 2018 Q1
Loading outpatient data for 2018 Q2
Loading outpatient data for 2018 Q3
Loading outpatient data for 2018 Q4
Loading outpatient data for 2019 Q1
Loading outpatient data for 2019 Q2
Loading outpatient data for 2019 Q3
Loading outpatient data for 2019 Q4
Loading outpatient data for 2020 Q1
Loading outpatient data for 2020 Q2
Loading outpatient data for 2020 Q3
Loading outpatient data for 2020 Q4
Loading outpatient data for 2021 Q1
Loading outpatient data for 2021 Q2
Loading outpatient data for 2021 Q3
Loading outpatient data for 2021 Q4
Loading outpatient data for 2022 Q1
Loading outpatient data for 2022 Q2
Loading outpatient data for 2022 Q3
Loading outpatient data for 2022 Q4
Loading outpatient data for 2023 Q1

In [8]:
import pandas as pd

mdf_2d = pd.read_parquet(
    os.path.join(root_dir, "./data/processed/hana_ent/mdf_2d.parquet")
)
mdf_3d = pd.read_parquet(
    os.path.join(root_dir, "./data/processed/hana_ent/mdf_3d.parquet")
)


In [11]:
og['count'] = 1
supply = og.pivot_table(
    index='date', 
    columns='prescription', 
    values='count', 
    aggfunc='count'
)
supply.fillna(0, inplace=True)

In [15]:
supply.to_parquet(
    os.path.join(root_dir, "./data/processed/hana_ent/supply.parquet")
)

In [13]:
mdf_2d

sex  age   umap1_x    umap2_x    umap1_y    umap2_y  \
id     date                                                              
1856   2018-03-16    0   81  3.148506  12.635462  23.636797  -7.886513   
       2019-04-01    0   82 -0.468371  12.208639   6.686179   8.219299   
2060   2018-03-19    0   79  0.263867  -7.967208  -2.284466  -9.089514   
2347   2018-03-09    1   84 -0.292361  13.326589  19.105474   5.849760   
2383   2019-02-08    1   82  0.767448   7.573028  23.632252  -7.882043   
...                ...  ...       ...        ...        ...        ...   
682613 2024-12-31    0   44  1.700175  12.265556   4.525751  11.978066   
682615 2024-12-31    0   73  0.143928  12.595835   5.962977  -8.346228   
682640 2024-12-31    0   13 -1.538649  10.466944   5.933376  -0.255164   
682642 2024-12-31    0   28 -1.085398  11.069419  10.486105   0.802031   
682655 2024-12-31    0   34  4.194273  11.257504   7.486858  -7.323721   

                       umap1      umap2  cluster_label  
id     date                                             
1856   2018-03-16  18.395563  -8.230709             -1  
       2019-04-01  18.635786  -8.641885              0  
2060   2018-03-19  18.495396  -8.317456             -1  
2347   2018-03-09   2.861417  11.572426             -1  
2383   2019-02-08   2.777116  10.387681             -1  
...                      ...        ...            ...  
682613 2024-12-31   2.499509   2.891868              0  
682615 2024-12-31   3.036672  12.032837              0  
682640 2024-12-31  18.742035  -8.660015              0  
682642 2024-12-31  18.705198  -8.051968              0  
682655 2024-12-31   0.066441  -3.688938              0  

[61594 rows x 9 columns]

In [14]:
og

,id,date,sex,age,primary_diagnosis,secondary_diagnosis,prescription,department,count
0,1856,2018-03-16,0,81,만성 인두염,"만성 후두염, 만성 편도염",tramadol,"Ear, Nose and Throat",1
1,1856,2018-03-16,0,81,만성 인두염,"만성 후두염, 만성 편도염",tramadol,"Ear, Nose and Throat",1
2,1856,2018-03-16,0,81,만성 인두염,"만성 후두염, 만성 편도염",tramadol,"Ear, Nose and Throat",1
3,1856,2019-04-01,0,82,만성 비염,"만성 범부비동염, 상세불명의 만성 부비동염, 출혈 또는 천공이 없는 급성 상세불명 ...",tramadol,"Ear, Nose and Throat",1
4,1856,2019-04-01,0,82,만성 비염,"만성 범부비동염, 상세불명의 만성 부비동염, 출혈 또는 천공이 없는 급성 상세불명 ...",tramadol,"Ear, Nose and Throat",1
...,...,...,...,...,...,...,...,...,...
369235,682655,2024-12-31,0,34,재발성으로 명시되어 있지 않은 부비동염,"기타 알레르기비염, 다년성 알레르기비염, 기타 위염",diclofenac,"Ear, Nose and Throat",1
369236,682655,2024-12-31,0,34,재발성으로 명시되어 있지 않은 부비동염,"기타 알레르기비염, 다년성 알레르기비염, 기타 위염",dexamethasone,"Ear, Nose and Throat",1
369237,682655,2024-12-31,0,34,재발성으로 명시되어 있지 않은 부비동염,"기타 알레르기비염, 다년성 알레르기비염, 기타 위염",dexamethasone,"Ear, Nose and Throat",1
369238,682655,2024-12-31,0,34,재발성으로 명시되어 있지 않은 부비동염,"기타 알레르기비염, 다년성 알레르기비염, 기타 위염",dexamethasone,"Ear, Nose and Throat",1
